In [1]:
import requests as req
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import json
import logging
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [17]:
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

tescoCNFUrl ="https://eshop.tesco.com.my/groceries/en-GB/shop/chilled-and-frozen/all?page={page}"
tescoFFUrl ="https://eshop.tesco.com.my/groceries/en-GB/shop/fresh-food/all?page={page}"

url = tescoCNFUrl.format(page=str(1))
res = req.get(url, headers=headers)
html = res.text

soup = BeautifulSoup(html, "html.parser")
lastPageElems = soup.find_all("li", class_="pagination-btn-holder")
lastPage = int(str(lastPageElems[len(lastPageElems)-2]).split('<span aria-hidden="true">')[1].split('</span>')[0])

In [3]:
# Scrape Chilled and Frozen Section
name = []; price = []; pUnit = []; link = []

for i in range(1,lastPage+1,1):
    url = tescoCNFUrl.format(page=str(i))
    res = req.get(url,headers =headers)
    html = res.text
    soup = BeautifulSoup(html,"html.parser")
    nameList = soup.find_all("a",class_="ui__StyledLink-sc-18aswmp-0 bfYkKW")
    priceList = soup.find_all("span",class_="value")

    for j in nameList:
        name.append(j.contents[0])      
        link.append("https://eshop.tesco.com.my"+j['href'])
        
    k=0
    while k<len(priceList):
        price.append(priceList[k].contents[0])
        pUnit.append(priceList[k+1].contents[0])
        k+=2

toDf = pd.DataFrame(zip(name,price,pUnit,link), columns=['Product Name','Product Price','Price per Unit','Product Link'])

In [24]:
toDf.head()

,Product Name,Product Price,Price per Unit,Product Link
0,Farm Fresh Pure Fresh Milk 2 Litres,14.50,14.50,https://eshop.tesco.com.my/groceries/en-GB/pro...
1,Farm Fresh Pure Fresh Milk 1 Litre,6.99,6.99,https://eshop.tesco.com.my/groceries/en-GB/pro...
2,Ayamas Chicken Frankfurters 340g,3.99,3.99,https://eshop.tesco.com.my/groceries/en-GB/pro...
3,Yakult Ace Cultured Milk Drinks 10 x 80ml,8.89,8.89,https://eshop.tesco.com.my/groceries/en-GB/pro...
4,Anchor Salted Pure New Zealand Butter 227g,12.35,12.35,https://eshop.tesco.com.my/groceries/en-GB/pro...
